# AR Visualization Pipeline Example: v2
An improved AR viz pipeline example. At this point, we've streamlined the process of creating the field skeleton and configuring the EBTEL simulations (or any hydro code we'd like to use) and then loading all of these results back into the field object. The next step is to figure out how to use all this information to calculate the emission in a sensible way. We'll explore that a bit here.

In [1]:
import os
import subprocess
import itertools
import pickle
import types
import copy
import glob

import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import seaborn as sns
import astropy.units as u
from astropy.io import fits
from sunpy.net import vso
import sunpy.cm
import yt
import h5py
from IPython.display import HTML
from IPython.display import Image

import synthesizAR
from synthesizAR.model_ext import UniformHeating,PowerLawScaledWaitingTimes,PowerLawUnscaledWaitingTimes
from synthesizAR.model_ext import EbtelInterface 

sns.set_context(context='notebook',font_scale=1.5)
%matplotlib inline

/opt/anaconda3/envs/synthesizar/lib/python3.5/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


# Field Skeleton

In [ ]:
client = vso.VSOClient()
result_hmi = client.query(
     vso.attrs.Time((2013, 1, 1, 7, 34, 0), (2013, 1, 1, 9, 0, 0)),
     vso.attrs.Instrument('HMI'),
     vso.attrs.Physobs('LOS_magnetic_field'),   # Physical observables
     vso.attrs.Sample(5000 * u.s)
)
data_hmi = client.get(result_hmi,methods=('URL-FILE_Rice','URL-FILE')).wait()

In [ ]:
crop = (u.Quantity([-140,90]*u.arcsec),u.Quantity([420,560]*u.arcsec))
resample = u.Quantity([100,100]*u.pixel)
field = synthesizAR.Skeleton(data_hmi[0],crop=crop,resample=resample)
zshape=50
zrange=u.Quantity([0.,150.]*u.arcsec)
field.extrapolate_field(zshape,zrange)

In [ ]:
field.extract_streamlines(500)

In [ ]:
field.peek(alpha=0.35)

# Loops/Hydro Simulations

In [ ]:
field.make_loops()

In [ ]:
heating_options = {
    'duration':200.0,
    'duration_rise':100.0,
    'duration_decay':100.0,
    'average_waiting_time':1250.0,
    'stress_level':0.3,
    'alpha':-2.5,
    'delta_power_law_bounds':10,
    'waiting_time_scaling':1.0,
}
uni_model = UniformHeating(heating_options)
pl_model = PowerLawUnscaledWaitingTimes(heating_options)
pl_scaled_model = PowerLawScaledWaitingTimes(heating_options)

In [ ]:
ih = synthesizAR.util.InputHandler(os.path.join(os.environ['RESEARCH_DIR'],'ebtelPlusPlus/config/ebtel.example.cfg.xml'))
base_config = ih.lookup_vars()
base_config['use_adaptive_solver'] = True
base_config['tau'] = 1.0
base_config['adaptive_solver_error'] = 1e-12
base_config['adaptive_solver_safety'] = 0.01
base_config['total_time'] = 5000.0
ebtel_plug = EbtelInterface(base_config,pl_scaled_model)
field.configure_loop_simulations(ebtel_plug,
                                 parent_config_dir='/data/datadrive2/ar_viz/test/config/',
                                 parent_results_dir='/data/datadrive2/ar_viz/test/results')

In [ ]:
for loop in field.loops:
    subprocess.call([os.path.join(os.environ['RESEARCH_DIR'],'ebtelPlusPlus/bin/ebtel++.run'),
                     '-c',loop.hydro_configuration['config_filename']])

Now load the simulation results, either loading into memory or writing to an HDF5 file to conserve some memory.

In [ ]:
field.load_loop_simulations(ebtel_plug,savefile='/data/datadrive2/ar_viz/test/loop_parameters.h5')

Take a look at the loop simulations

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(10,8),sharex=True)
for l in field.loops:
    ax[0].plot(l.time,l.temperature[:,0],color=sns.color_palette('deep')[0],alpha=0.08)
    ax[1].plot(l.time,l.density[:,0],color=sns.color_palette('deep')[2],alpha=0.08)
ax[0].set_ylabel(r'$T$ ({0})'.format(l.temperature.unit.to_string()))
ax[1].set_ylabel(r'$n$ ({})'.format(l.density.unit))
ax[1].set_xlabel(r'$t$ ({})'.format(l.time.unit))

## Calculate Emissivity

First, we construct an emissivity model that takes temperature and density and calculates emissivity. This can be passed to the field object and then is calculated for each loop. These are passed to the loop objects and saved in an HDF5 file (if savefile is set) to avoid storing all of this in memory.

Make a sample emissivity model object. This will be how we configure the emissivity calculation in the package.

In [ ]:
class SimpleAIAEmissModel(object):
    def __init__(self,tresponse_filename,channels=['94','131','171','193','211','335']):
        self.channels = channels
        self.temperature_response = {}
        _tmp = np.loadtxt(tresponse_filename)
        self.temperature = 10**_tmp[:,0]
        for c in channels:
            self.temperature_response[c] = _tmp[:,channels.index(c)+1]
        self._configure_interpolators()
            
    def _configure_interpolators(self):
        self.interpolators = {}
        for c in self.channels:
            self.interpolators[c] = interpolate.interp1d(self.temperature,self.temperature_response[c],kind='linear')
            
    def calculate_emissivity(self,temperature,density):
        emissivity = {}
        for c in self.channels:
            emissivity[c] = np.reshape(np.ravel(density**2)*self.interpolators[c](np.ravel(temperature)),
                                       np.shape(density)
                                      )*(u.cm)**5/u.s/u.pixel*u.count
        return emissivity


Now, instantiate the model and then pass it to the field.

In [ ]:
aia_emissivity = SimpleAIAEmissModel('aia_tresponse_raw.dat')

In [ ]:
field.calculate_emissivity(aia_emissivity,savefile='/data/datadrive2/ar_viz/test/loop_parameters.h5')

Save the field here.

In [ ]:
field.save_field(savedir='/data/datadrive2/ar_viz/test/')

Reload field here.

In [ ]:
field = synthesizAR.Skeleton.restore_field('/data/datadrive2/ar_viz/test/synthesizAR-field-save_20161026-171009/')

## Constructing the Observation/Instrument Data Product

We want to avoid loading a bunch of stuff into memory. Otherwise, we'll quickly run into scalability issues. The loops will be loaded into a 3D histogram and then flattened to a 2D image. This image is then saved as a SunPy map with the appropriate header information.

First, setup the SunPy header information.

In [ ]:
synth_header = sunpy.map.header.MapMeta()
replacements = ['crpix1','crpix2','crval1','crval2','cdelt1','cdelt2','cunit1','cunit2','crlt_obs','ctype1',
                'ctype2','date-obs','dsun_obs','rsun_obs']
for r in replacements:
    synth_header[r] = field.clipped_hmi_map.meta[r]
synth_header['naxis1'] = int(field.clipped_hmi_map.dimensions.x.value)
synth_header['naxis2'] = int(field.clipped_hmi_map.dimensions.y.value)
synth_header['datavals'] = int(field.clipped_hmi_map.dimensions.x.value*field.clipped_hmi_map.dimensions.y.value)
synth_header['telescop'] = 'SDO/AIA'
synth_header['detector'] = 'AIA'
synth_header['waveunit'] = 'angstrom'
synth_header['instrume'] = 'AIA_4'

Interpolating at every time step is way too slow. Instead what we have to do is print to file an interpolated emissivity matrix (in $s$ and $t$) for each $\lambda$ for each loop. Each loop will have its own HDF5 file. These can than be easily sliced at the appropriate time index during the binning phase. This way, the interpolation is done in the most efficient way possible and nothing is stored in memory.

Now combine all of the loop emissivity information at each timestep at the appropriate resolution.

In [ ]:
file_template = os.path.join('/data/datadrive2/ar_viz/test/sunpy_fits',
                             '{wave}_channel','sunpy_map_t{time:04d}.fits')
#choose the first 10 seconds of the AR evolution
max_time=4.99e+3#9.99e+3
cadence=1.0
global_time = np.arange(0.0,max_time,cadence)
observing_time = global_time[::50]
ds = field._convert_angle_to_length(0.3*u.arcsec)
total_interpolated_s_points = sum([int(np.ceil(loop.full_length/ds)) for loop in field.loops])
#create a new HDF5 file for storing each of these datasets.
with h5py.File('/data/datadrive2/ar_viz/test/detector_counts.h5','w') as hf:
    for c in aia_emissivity.channels:
        hf.create_dataset(c,(len(observing_time),total_interpolated_s_points))

In [ ]:
#iterate over loops
start_index=0
total_coordinates = []
for loop in field.loops:
    ##
    N_interp = int(np.ceil(loop.full_length/ds))
    interpolated_s = np.linspace(loop.field_aligned_coordinate[0].value,
                                 loop.field_aligned_coordinate[-1].value,N_interp)
    nots,_ = interpolate.splprep(loop.coordinates.value.T)
    _tmp = interpolate.splev(np.linspace(0,1,N_interp),nots)
    _tmp_coords = [(x,y,z) for x,y,z in zip(_tmp[0],_tmp[1],_tmp[2])]
    total_coordinates += _tmp_coords
    ##
    with h5py.File('/data/datadrive2/ar_viz/test/detector_counts.h5','a') as hf:
        for c in aia_emissivity.channels:
            f_s = interpolate.interp1d(loop.field_aligned_coordinate.value,loop.get_emissivity(c).value,axis=1)
            #in real life, will need to combine several different wavelengths into a channel.
            #this step may be a bit more computationally intensive in the end.
            emiss = interpolate.interp1d(loop.time.value,
                                         f_s(interpolated_s),axis=0)(observing_time)
            dset = hf[c]
            dset[:,start_index:(start_index+N_interp)] = emiss
    start_index += N_interp
            

Set up the bin edges for the histograms.

In [ ]:
min_z = min(field.extrapolated_3d_field.domain_left_edge[2].value,np.array(total_coordinates)[:,2].min())
max_z = max(field.extrapolated_3d_field.domain_right_edge[2].value,np.array(total_coordinates)[:,2].max())
z_edges = np.append(np.arange(min_z,max_z,field._convert_angle_to_length(0.6*u.arcsec).value),
                    max_z)
bins = [int(field.clipped_hmi_map.dimensions.x.value),
        int(field.clipped_hmi_map.dimensions.y.value),
        len(z_edges)-1]
bin_ranges=[field._convert_angle_to_length(field.clipped_hmi_map.xrange).value,
            field._convert_angle_to_length(field.clipped_hmi_map.yrange).value,
            [z_edges[0],z_edges[-1]]]

Bin all of the combined loop data at every timestep for every channel and project down to the $x-y$ plane, printing as a FITS file.

In [ ]:
with h5py.File('/data/datadrive2/ar_viz/test/detector_counts.h5','r') as hf:
    for c in aia_emissivity.channels:
        dset = hf[c]
        for i,time in enumerate(observing_time):
            tmp = np.array(dset[i,:])
            hist,_ = np.histogramdd(np.array(total_coordinates),
                                    bins=bins,
                                    range=bin_ranges,
                                    weights=tmp)
            synth_header['wavelnth'] = int(c)
            synth_header['t_obs'] = time
            synth_map = sunpy.map.Map(np.dot(hist,np.diff(z_edges)).T,synth_header)
            synth_map = synth_map.resample(
            u.Quantity([((field.clipped_hmi_map.xrange[1]-field.clipped_hmi_map.xrange[0])/(0.6*u.arcsec)).value,
                ((field.clipped_hmi_map.yrange[1]-field.clipped_hmi_map.yrange[0])/(0.6*u.arcsec)).value])*u.pixel)
            
            synth_map.save(file_template.format(wave=c,time=i))

### Passing the Data to SunPy
Now, let's make some movies using SunPy maps.

In [ ]:
height = 6
width = 2.55*height
fig,axes = plt.subplots(2,3,figsize=(width,height),sharex=True,sharey=True)
for j in range(len(observing_time)):
    fig.suptitle(r'$t=${0} ({1})'.format(observing_time[j],u.s),fontsize=16)
    for c,ax in zip(aia_emissivity.channels,axes.flatten()):
        _tmp_map = sunpy.map.Map(file_template.format(wave=c,time=j))
        _tmp_map.plot(axes=ax,vmin=1.0,vmax=6e+3,annotate=False)
        #plt.colorbar(ax=ax)
        ax.set_title(r'{0} $\mathrm{{\mathring{{A}}}}$'.format(c))
    for i in range(2):  axes[i,0].set_ylabel(r'$y$ ({})'.format(_tmp_map.yrange.unit))
    for i in range(3):  axes[1,i].set_xlabel(r'$x$ ({})'.format(_tmp_map.xrange.unit))
    plt.savefig('/data/datadrive2/ar_viz/test/sunpy_fits/all_channels/all_channels_{time:04d}.pdf'.format(time=j))
fig.clf()

Make a gif from these PDFs.

In [ ]:
%%bash
convert -delay 10 -loop 0 /data/datadrive2/ar_viz/test/sunpy_fits/all_channels/*.pdf aia_all_channels.gif

In [ ]:
HTML('<img src="aia_all_channels.gif"/>')

## Extra `yt` Stuff
Some of this will get incorporated into the observe function.

Now to see if this all worked, try to create a yt dataset and visualize the density of all loops. Make a function to build and save a multipanel figure

In [ ]:
def build_plots(ds, proj='z', channels=['sdoaia94','sdoaia131','sdoaia171','sdoaia193','sdoaia211','sdoaia335'], 
                fn=None, zlim_min=1e-3, zlim_max=100):
    #set up matplotlib canvas
    fig = plt.figure()
    grid = AxesGrid(fig, (0.075,0.075,0.85,0.85),
                    nrows_ncols = (2, 3),
                    axes_pad = 1.2,
                    label_mode = "1",
                    share_all = True,
                    cbar_location="right",
                    cbar_mode="each",
                    cbar_size="3%",
                    cbar_pad="0%")
    #set up yt plotting stuff
    pproj = yt.ProjectionPlot(ds,proj,channels)
    pproj.set_zlim(channels,zlim_min,zlim_max)
    pproj.set_figure_size(20)
    #pproj.annotate_timestamp()
    #configure colorbar and labels
    for c in channels:
        _tmp = sunpy.cm.get_cmap(c)
        _tmp.set_bad(_tmp(0))
        pproj.set_cmap(c,_tmp)
        pproj.set_colorbar_label(c,'AIA {0} $\mathring{{\mathrm{{A}}}}$'.format(c[6:]))
    #make the plot
    for i,c in enumerate(channels):
        plot = pproj.plots[c]
        plot.figure = fig
        plot.axes = grid[i].axes
        plot.cax = grid.cbar_axes[i]
    #draw and save
    pproj._setup_plots()
    if fn is not None:
        plt.savefig(fn)

Create a function for creating a `yt` dataset.

In [ ]:
def build_dataset(ar,time_index):
    #get current time
    cur_time = ar.loops[0].time[time_index]
    #flatten density,temperature arrays
    density_flat = np.array([n for l in ar.loops for n in l.density.value[time_index,:]])
    temperature_flat = np.array([T for l in ar.loops for T in l.temperature.value[time_index,:]])
    #calculate emission
    aia94_emiss = (density_flat**2)*f_aia94(temperature_flat)
    aia131_emiss = (density_flat**2)*f_aia171(temperature_flat)
    aia171_emiss = (density_flat**2)*f_aia131(temperature_flat)
    aia193_emiss = (density_flat**2)*f_aia193(temperature_flat)
    aia211_emiss = (density_flat**2)*f_aia211(temperature_flat)
    aia335_emiss = (density_flat**2)*f_aia335(temperature_flat)
    #load dataset
    data = dict(
        sdoaia94=(np.histogramdd(coords_flat,bins=bins,weights=aia94_emiss)[0],"1/cm/s"),
        sdoaia131=(np.histogramdd(coords_flat,bins=bins,weights=aia131_emiss)[0],"1/cm/s"),
        sdoaia171=(np.histogramdd(coords_flat,bins=bins,weights=aia171_emiss)[0],"1/cm/s"),
        sdoaia193=(np.histogramdd(coords_flat,bins=bins,weights=aia193_emiss)[0],"1/cm/s"),
        sdoaia211=(np.histogramdd(coords_flat,bins=bins,weights=aia211_emiss)[0],"1/cm/s"),
        sdoaia335=(np.histogramdd(coords_flat,bins=bins,weights=aia335_emiss)[0],"1/cm/s"),
    )
    bbox = np.array([
            np.array([bin_x_edges[0],bin_x_edges[-1]]),
            np.array([bin_y_edges[0],bin_y_edges[-1]]),
            np.array([bin_z_edges[0],bin_z_edges[-1]]),
        ])
    ds = yt.load_uniform_grid(data,data['sdoaia94'][0].shape,bbox=bbox,
                              length_unit='cm',geometry=('cartesian',('x','y','z')))
    ds.current_time = cur_time*yt.units.second
    return ds

In [ ]:
ds0 = build_dataset(field,0)

In [ ]:
build_plots(build_dataset(field,200),zlim_max=1000,zlim_min=1e-3)

Now, let's try saving  all of this data as 3d structures.

In [ ]:
for i in range(len(field.loops[0].time.value[::10])):
    print('Processing frame at time {}'.format(t))
    ds = build_dataset(field,i)
    build_plots(ds,fn='/data/datadrive2/ar_viz/test/frames/frame_{}.pdf'.format(i))
    ds.close()
    #cg = ds.covering_grid(level=0,left_edge=ds.domain_left_edge,dims=ds.domain_dimensions)
    #cg.save_as_dataset(filename='/data/datadrive2/ar_viz/test/structures_3d/ar{}.h5'.format(i),
    #                   fields=[fie[1] for fie in ds.field_list]
    #                  )

## Query AIA Image

In [ ]:
client = vso.VSOClient()

In [ ]:
result_aia = client.query(
    vso.attrs.Time((2013, 1, 1, 7, 34, 0), (2013, 1, 1, 9, 0, 0)), # Time range.
    vso.attrs.Instrument('AIA'),
    vso.attrs.Physobs('intensity'), 
    vso.attrs.Sample(5000 * u.s)    
)
data_aia = client.get(result_aia, methods=('URL-FILE_Rice', 'URL-FILE')).wait()

In [ ]:
aia_map = sunpy.map.Map(data_aia[1])

In [ ]:
aia_map.meta['cdelt1']

In [ ]:
aia_map.xrange

In [ ]:
aia_map.meta.has_key('btype')